## Final Project - Group 3 - Credit Card Fraud Detection
Project for UCB extension Machine learning and deep learning in Spark

### Kaggle - Credit Card Fraud Detection
https://www.kaggle.com/mlg-ulb/creditcardfraud
#### Context
It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.
#### Content
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.
#### Inspiration
Identify fraudulent credit card transactions.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.
#### Acknowledgements
The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML

Please cite: Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

In [2]:
%sh
#need to run ***ONCE*** to install SMOTE package
/home/ubuntu/databricks/python/bin/pip install 'imbalanced-learn<0.2.1'
pip freeze | grep imbalanced-learn

In [3]:
# File location and type
file_location = "/FileStore/tables/creditcard.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [4]:
# Imported Libraries
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [5]:
df.printSchema()

In [6]:
df.describe('Amount').show()

In [7]:
# number of null or nan values
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [8]:
# The classes are heavily skewed we need to solve this issue later.
print 'Number of frauds: ', df.filter(F.col('Class') == 1).count(), ", ", round((df.filter(F.col('Class') == 1).count())/float(df.count()) * 100, 3), '% of the dataset'
print 'Number of records', df.count()

In [9]:
display(df.groupBy('class').count())

In [10]:
display(df)

In [11]:
display(df)

In [12]:
df_fraud = df.filter(F.col('Class') == 1)
df_no_fraud_sample = df.filter(F.col('Class') == 0).sample(False, 0.1).limit(4920) #downsaple no fraud class 

df_sample = df_fraud.union(df_no_fraud_sample)
display(df_sample)

In [13]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import RFormula


rf = RFormula(formula="~ V1 + V2 + V3 + V4 + V5 + V6 + V7 + V8 + V9 + V10 + V11 + V12 + V13 + V14 + V15 + V16 + V17 + V18 + V19 + V20 + V21 + V22 + V23 + V24 + V25 + V26 + V27")
final_df_rf = rf.fit(df_sample).transform(df_sample)

pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(final_df_rf)

result = model.transform(final_df_rf).select("pcaFeatures")
import numpy as np
df_pca_arr = np.array(result.collect())
df_pca_no_class = sc.parallelize(df_pca_arr[:,0]).map(lambda x: x.tolist()).toDF(['V1', 'V2'])
df_pca_no_class = df_pca_no_class.withColumn("id", F.monotonically_increasing_id())
df_pca_no_class.createOrReplaceTempView('df_pca_no_class')
df_pca_no_class = spark.sql('select row_number() over (order by "id") as num, * from df_pca_no_class')
df_pca_class = df_sample.select('class')
df_pca_class = df_pca_class.withColumn("id", F.monotonically_increasing_id())
df_pca_class.createOrReplaceTempView('df_pca_class')
df_pca_class = spark.sql('select row_number() over (order by "id") as num, * from df_pca_class')

df_pca = df_pca_no_class.join(df_pca_class, "num", 'inner').drop("id")
display(df_pca)

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Weighted Logistic Regression Approach

In [16]:
# Calculate the balancing ratio for the dataset
# balancing_ratio = NumberPositives / TotalRecords
balancing_ratio = float(492) / float(284807)
df_weighted = df.withColumn("weight", F.when(F.col('Class') == 0, balancing_ratio)\
                           .otherwise(1 - balancing_ratio))

## Resampling
We undersampling the normal trascation

In [18]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
print df_fraud.count()
df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(df_fraud.count()) #downsaple no fraud class 
print df_no_fraud_sample.count()

df_train_sample = df_fraud.union(df_no_fraud_sample)
print df_train_sample.count()

In [19]:
def confusionmatrix(predictions):
  if 'Class' in predictions.columns:
    tp = predictions[(predictions.Class == 1) & (predictions.prediction == 1)].count()
    tn = predictions[(predictions.Class == 0) & (predictions.prediction == 0)].count()
    fp = predictions[(predictions.Class == 0) & (predictions.prediction == 1)].count()
    fn = predictions[(predictions.Class == 1) & (predictions.prediction == 0)].count()
  else:
    tp = predictions[(predictions.label == 1) & (predictions.prediction == 1)].count()
    tn = predictions[(predictions.label == 0) & (predictions.prediction == 0)].count()
    fp = predictions[(predictions.label == 0) & (predictions.prediction == 1)].count()
    fn = predictions[(predictions.label == 1) & (predictions.prediction == 0)].count()    

  print "True Positives:", tp
  print "True Negatives:", tn
  print "False Positives:", fp
  print "False Negatives:", fn
  print "Total", predictions.count()

  r = float(tp)/(tp + fn)
  print "recall", r

  p = float(tp) / (tp + fp)
  print "precision", p

In [20]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.mllib.evaluation import BinaryClassificationMetrics

def data_prep_train(trainDF):
  
  # produce train and test dataframe objects
  featureAssembler = VectorAssembler()\
  .setInputCols(["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12",\
                                                     "V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"])\
    .setOutputCol("features")  

  rf = RandomForestClassifier()\
    .setLabelCol("Class")\
    .setFeaturesCol("features")\
    .setNumTrees(100)\
    .setMaxBins(32)\
    .setMaxDepth(10)

  rfpipeline = Pipeline()\
    .setStages([featureAssembler, rf])

  rfmodel=rfpipeline.fit(trainDF)
  return rfmodel

In [21]:
# Try random forest for classification
rfmodel = data_prep_train(df_train_sample)

# Make predictions
predictions = rfmodel.transform(df_test)


evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

In [23]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
print "Fraud count:", df_fraud.count()

for fraud_ratio in range (1,20):
  print "---------- Predicitions #", fraud_ratio, "----------"
  df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(df_fraud.count()*fraud_ratio) #downsaple no fraud class 
  print "No fraud count:", df_no_fraud_sample.count()
  
  df_train_sample = df_fraud.union(df_no_fraud_sample)
  print "Total record count:", df_train_sample.count()
  
  # Try random forest for classification
  rfmodel = data_prep_train(df_train_sample)

  # Make predictions
  predictions = rfmodel.transform(df_test)


  evaluator = MulticlassClassificationEvaluator()\
    .setLabelCol("Class")\
    .setPredictionCol("prediction")\
    .setMetricName("accuracy")
  accuracy = evaluator.evaluate(predictions)

  print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
  confusionmatrix(predictions)

In [24]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
no_fraud_count = df_train.filter(F.col('Class') == 0).count()
print "Fraud count:", df_fraud.count()

for fraud_ratio in range (2,5):
  print "---------- Predicitions #", fraud_ratio, "----------"
  df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(no_fraud_count/fraud_ratio) #downsaple no fraud class 
  print "No fraud count:", df_no_fraud_sample.count()
  
  df_train_sample = df_fraud.union(df_no_fraud_sample)
  print "Total record count:", df_train_sample.count()
  
  # Try random forest for classification
  rfmodel = data_prep_train(df_train_sample)

  # Make predictions
  predictions = rfmodel.transform(df_test)


  evaluator = MulticlassClassificationEvaluator()\
    .setLabelCol("Class")\
    .setPredictionCol("prediction")\
    .setMetricName("accuracy")
  accuracy = evaluator.evaluate(predictions)

  print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
  confusionmatrix(predictions)

In [25]:
# Continuing where the last cell stopped:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
print "Fraud count:", df_fraud.count()

for fraud_ratio in range (8,20):
  print "---------- Predicitions #", fraud_ratio, "----------"
  df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(df_fraud.count()*fraud_ratio) #downsaple no fraud class 
  print "No fraud count:", df_no_fraud_sample.count()
  
  df_train_sample = df_fraud.union(df_no_fraud_sample)
  print "Total record count:", df_train_sample.count()
  
  # Try random forest for classification
  rfmodel = data_prep_train(df_train_sample)

  # Make predictions
  predictions = rfmodel.transform(df_test)


  evaluator = MulticlassClassificationEvaluator()\
    .setLabelCol("Class")\
    .setPredictionCol("prediction")\
    .setMetricName("accuracy")
  accuracy = evaluator.evaluate(predictions)

  print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
  confusionmatrix(predictions)

In [26]:
# Continuing where the last cell stopped:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
print "Fraud count:", df_fraud.count()

fraud_ratio = 20
print "---------- Predicitions #", fraud_ratio, "----------"
df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(df_fraud.count()*fraud_ratio) #downsaple no fraud class 
print "No fraud count:", df_no_fraud_sample.count()

df_train_sample = df_fraud.union(df_no_fraud_sample)
print "Total record count:", df_train_sample.count()

# Try random forest for classification
rfmodel = data_prep_train(df_train_sample)

# Make predictions
predictions = rfmodel.transform(df_test)


evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

Created graph of results in excel

In [28]:
# Keep cluster running

In [29]:
# Continuing where the last cell stopped:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Try random forest for classification
rfmodel = data_prep_train(df_train)

# Make predictions
predictions = rfmodel.transform(df_test)


evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

In [30]:
from pyspark.ml.classification import LogisticRegression

def data_prep_train(trainDF):
  
  # produce train and test dataframe objects
  featureAssembler = VectorAssembler()\
  .setInputCols(["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12",\
                                                     "V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"])\
    .setOutputCol("features")  

  lr = LogisticRegression(maxIter=10, regParam=0.001)\
    .setLabelCol("Class")\
    .setFeaturesCol("features")

  lrpipeline = Pipeline()\
    .setStages([featureAssembler, lr])

  lrmodel=lrpipeline.fit(trainDF)
  
  return lrmodel

In [31]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Try random forest for classification
lrmodel = data_prep_train(df_train)

# Make predictions
predictions = lrmodel.transform(df_test)


evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

In [32]:
mean_Amount, sttdev_Amount = df.select(F.mean("Amount"), F.stddev("Amount")).first()
df_scaled = df.withColumn("Amount_scaled", (F.col("Amount") - mean_Amount) / sttdev_Amount)
df_scaled = df_scaled.drop('Time','Amount')
display(df_scaled)

In [33]:
def data_prep_train_scaled(trainDF):
  
  # produce train and test dataframe objects
  featureAssembler = VectorAssembler()\
  .setInputCols(["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12",\
                                                     "V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount_scaled"])\
    .setOutputCol("features")  

  rf = RandomForestClassifier()\
    .setLabelCol("Class")\
    .setFeaturesCol("features")\
    .setNumTrees(100)\
    .setMaxBins(32)\
    .setMaxDepth(10)

  rfpipeline = Pipeline()\
    .setStages([featureAssembler, rf])

  rfmodel=rfpipeline.fit(trainDF)
  return rfmodel

In [34]:
import pandas as pd
from imblearn.over_sampling import SMOTE,RandomOverSampler
from sklearn.model_selection import train_test_split
from collections import Counter
df_pd = df.toPandas()

#df_train, df_test = df.randomSplit([0.7, 0.3], 42)
#df_train_pd = df_train.toPandas()

X = df_pd.iloc[:, df_pd.columns != 'Class']
Y = df_pd.iloc[:, df_pd.columns == 'Class']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

sm = SMOTE(random_state=12, ratio = 'auto', kind = 'regular')

x_train_res, y_train_res = sm.fit_sample(x_train, y_train)
print 'Resampled dataset shape {}'.format(Counter(y_train_res))

In [35]:
xcol = df.columns
ycol = [xcol.pop()]
df_pd_res_x = pd.DataFrame(x_train_res, columns=xcol)
df_pd_res_y = pd.DataFrame(y_train_res, columns=ycol)
df_pd_res = pd.concat([df_pd_res_x,df_pd_res_y], axis=1)
df_pd_res.head()

In [36]:
df_pd_x_test = pd.DataFrame(x_test, columns=xcol)
df_pd_y_test = pd.DataFrame(y_test, columns=ycol)
df_pd_test = pd.concat([df_pd_x_test,df_pd_y_test], axis=1)
df_pd_test.head()

In [37]:
df_train_res = spark.createDataFrame(df_pd_res)
df_test = spark.createDataFrame(df_pd_test)

In [38]:
# Try random forest for classification
rfmodel = data_prep_train(df_train_res)

# Make predictions
predictions = rfmodel.transform(df_test)

evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

In [39]:
mean_Amount, sttdev_Amount = df.select(F.mean("Amount"), F.stddev("Amount")).first()
df_scaled = df.withColumn("Amount_scaled", (F.col("Amount") - mean_Amount) / sttdev_Amount)
df_scaled = df_scaled.drop('Time','Amount')

df_pd = df_scaled.toPandas()

X = df_pd.iloc[:, df_pd.columns != 'Class']
Y = df_pd.iloc[:, df_pd.columns == 'Class']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

sm = SMOTE(random_state=12, ratio = 'auto', kind = 'regular')

x_train_res, y_train_res = sm.fit_sample(x_train, y_train)

xcol = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount_scaled"]
ycol = ["Class"]
df_pd_res_x = pd.DataFrame(x_train_res, columns=xcol)
df_pd_res_y = pd.DataFrame(y_train_res, columns=ycol)
df_pd_res = pd.concat([df_pd_res_x,df_pd_res_y], axis=1)

df_pd_x_test = pd.DataFrame(x_test, columns=xcol)
df_pd_y_test = pd.DataFrame(y_test, columns=ycol)
df_pd_test = pd.concat([df_pd_x_test,df_pd_y_test], axis=1)

df_train_res = spark.createDataFrame(df_pd_res)
df_test = spark.createDataFrame(df_pd_test)

# Try random forest for classification
rfmodel = data_prep_train_scaled(df_train_res)

# Make predictions
predictions = rfmodel.transform(df_test)

evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

In [40]:
# DOES NOT WORK IN SPARK
from imblearn.over_sampling import SMOTE,RandomOverSampler

df_train, df_test = df.randomSplit([0.7, 0.3], 42)

xcol = df_train.columns
ycol = xcol.pop()

df_train_x = df_train.select(xcol) # Feature columns
df_train_y = df_train.select(ycol) # Class column


df_train_x_array =  np.array(df_train_x.select(df_train_x.columns).collect())
df_train_y_array =  np.array(df_train_y.select(df_train_y.columns).collect())

sm = SMOTE(random_state=12, ratio = 'auto', kind = 'regular')
X_resampled, y_resampled = sm.fit_sample(df_train_x_array, df_train_y_array)

In [41]:
df_X_resampled = sc.parallelize(X_resampled).toDF(df_train_x.columns)
#X_resampled

### Model selection (a.k.a. hyperparameter tuning)
An important task in ML is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LogisticRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately.
https://spark.apache.org/docs/latest/ml-tuning.html

In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [44]:
# produce train and test dataframe objects
featureAssembler = VectorAssembler()\
 .setInputCols(["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"])\
  .setOutputCol("features")  

rf_cv = RandomForestClassifier()\
  .setLabelCol("label")\
  .setFeaturesCol("features")\
  .setNumTrees(10)\
  .setMaxBins(50)\
  .setMaxDepth(20)\

pipeline_cv = Pipeline(stages=[featureAssembler,rf_cv])

trainDF_cv = trainDF.withColumnRenamed('Class', 'label')
testDF_cv = testDF.withColumnRenamed('Class', 'label')

paramGrid = ParamGridBuilder() \
    .addGrid(rf_cv.setNumTrees, [75, 100, 125]) \
    .addGrid(rf_cv.setMaxBins, [20, 32, 45])\
    .addGrid(rf_cv.setMaxDepth, [8, 10, 15])\
    .build()

crossval = CrossValidator(estimator=pipeline_cv,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)

cvModel = crossval.fit(trainDF_cv)

In [45]:
prediction = cvModel.transform(testDF_cv)

evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("label")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(prediction)

print "Accuracy = %5.2f%%" % ((accuracy)*100)

confusionmatrix(prediction)

In [46]:
#make prediction on the unsee imbalanced dataset 
im_prediction = cvModel.transform(im_test)
confusionmatrix(im_prediction)

try slightly imbalance data for trining

In [48]:
df_weighted_train, im_test = df_weighted.randomSplit([0.9, 0.1], 42)

# Random Undersampling
df_fraud = df_weighted_train.filter(F.col('Class') == 1)
print df_fraud.count()
df_no_fraud_sample = df_weighted_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(3*df_fraud.count()) #downsaple no fraud class 
print df_no_fraud_sample.count()

df_weighted_sample = df_fraud.union(df_no_fraud_sample)
print df_weighted_sample.count()
# Random Oversampling


# First split the data without sampling for comparison
splits = df_weighted_sample.randomSplit([0.7, 0.3], 42) # 70/30 split for training/testing
(trainDF, testDF) = (splits[0], splits[1])

In [49]:
# Try random forest for classification
rfmodel = data_prep_train(trainDF)

# Make predictions
predictions = rfmodel.transform(testDF)

evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

In [50]:
im_prediction = rfmodel.transform(im_test)
confusionmatrix(im_prediction)